# Контрольная работа
## Выполнил: студент ПИ19-3 Ковалев А. И.

## models.py

In [6]:
from sqlalchemy import (
    Column,
    Integer,
    String,
    DateTime,
    Float,
    ForeignKey,
    CheckConstraint
)
from sqlalchemy.ext.declarative import as_declarative, declared_attr

__all__ = [
    'Base',
    'City',
    'ExamMark',
    'Lecturer',
    'Student',
    'Subject',
    'SubjectLecturer',
    'University'
]

CASCADE = "CASCADE"
SHORT_VARCHAR = 80
MIDDLE_VARCHAR = 255


@as_declarative()
class Base:
    id = Column(Integer, primary_key=True, autoincrement=True)

    @declared_attr
    def __tablename__(self):
        table_name = ''
        ascii_upper = range(ord('A'), ord('Z') + 1)
        for i, s in enumerate(self.__name__):
            if ord(s) in ascii_upper and i != 0:
                table_name += '_'
            table_name += s.upper()

        return table_name + 'S'

    def __repr__(self):
        field_strings = []
        for attr in self.__repr_attrs__:
            field_strings.append(f'{attr}={getattr(self, attr)}')

        return f"<{self.__class__.__name__} ({', '.join(field_strings)})>"

    __str__ = __repr__
    __repr_attrs__ = ['id']


class City(Base):
    name = Column(String(SHORT_VARCHAR))

    __tablename__ = "CITIES"
    __repr_attrs__ = ['name']


class ExamMark(Base):
    mark = Column(Integer)
    date = Column(DateTime)
    student_id = Column(Integer, ForeignKey("STUDENTS.id", ondelete=CASCADE))
    subject_id = Column(Integer, ForeignKey("SUBJECTS.id", ondelete=CASCADE))

    __repr_attrs__ = ['mark', 'date', 'student_id', 'subject_id']


class Lecturer(Base):
    first_name = Column(String(SHORT_VARCHAR))
    last_name = Column(String(SHORT_VARCHAR))
    city = Column(String(SHORT_VARCHAR))
    university_id = Column(Integer, ForeignKey("UNIVERSITIES.id", ondelete=CASCADE))

    __repr_attrs__ = ['first_name', 'last_name', 'city', 'university_id']


class Student(Base):
    first_name = Column(String(SHORT_VARCHAR))
    last_name = Column(String(SHORT_VARCHAR))
    stipend = Column(Float, CheckConstraint("stipend >= 0.00", name="stipend_is_positive"))
    course = Column(Integer)
    city = Column(String(SHORT_VARCHAR))
    birthdate = Column(DateTime)
    university_id = Column(Integer, ForeignKey("UNIVERSITIES.id", ondelete=CASCADE))

    __repr_attrs__ = ['first_name', 'last_name', 'stipend', 'course', 'city', 'birthdate', 'university_id']


class SubjectLecturer(Base):
    lecturer_id = Column(Integer, ForeignKey("LECTURERS.id", ondelete=CASCADE))
    subject_id = Column(Integer, ForeignKey("SUBJECTS.id", ondelete=CASCADE))

    __tablename__ = "SUBJECTS_LECTURERS"
    __repr_attrs__ = ['lecturer_id', 'subject_id']


class Subject(Base):
    name = Column(String(SHORT_VARCHAR), index=True)
    hour = Column(Integer, index=True)
    semester = Column(Integer)

    __repr_attrs__ = ['name', 'hour', 'semester']


class University(Base):
    name = Column(String(SHORT_VARCHAR))
    rating = Column(Integer)
    city = Column(String(SHORT_VARCHAR))

    __tablename__ = "UNIVERSITIES"
    __repr_attrs__ = ['name', 'rating', 'city']

## database.py

In [11]:
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy.orm.session import sessionmaker, Session


class DataBase:
    DATABASE = "sqlite"
    DB_NAME = "resources/students.db"
    engine: Engine = create_engine(f"{DATABASE}:///{DB_NAME}", echo=False)
    session: Session = sessionmaker(bind=engine)()

        
# Base.metadata.create_all(DataBase.engine)

## utils.py

In [17]:
import re


def task(n):
    def wrapper(function):
        def decorator(*args, **kwargs):
            print(str(n) + ". " + re.sub(r' +', ' ', function.__doc__.strip()) + "\n")
            result = function(*args, **kwargs)
#             print('\n\n')
            return result
        return decorator
    return wrapper

## queries.py

In [18]:
from sqlalchemy.orm import Query
from sqlalchemy.sql import or_, func
from datetime import datetime
from pprint import pprint

db = DataBase()

In [19]:
@task("2")
def second():
    """
    Напишите запрос, позволяющий получить из таблицы exam_marks
    значения столбца mark (экзаменационная оценка) для всех студентов,
    исключив из списка повторение одинаковых строк.
    """
    query: Query = db.session.query(
        ExamMark.mark
    ).distinct()
    pprint(query.all())
    
    
second()

2. Напишите запрос, позволяющий получить из таблицы exam_marks
 значения столбца mark (экзаменационная оценка) для всех студентов,
 исключив из списка повторение одинаковых строк.

[(1,), (4,), (3,), (5,), (2,), (None,)]


In [20]:
@task("3")
def third():
    """
    Напишите запрос для получения списка студентов
    без определенного места жительства.
    """
    query: Query = db.session.query(
        Student
    ).filter(
        Student.city.is_(None)
    )
    pprint(query.all())
    
    
third()

3. Напишите запрос для получения списка студентов
 без определенного места жительства.

[<Student (first_name=Ирина, last_name=Медведева, stipend=100.0, course=1, city=None, birthdate=2000-08-22 00:00:00, university_id=15)>,
 <Student (first_name=Оксана, last_name=Афанасьева, stipend=250.0, course=1, city=None, birthdate=2000-11-16 00:00:00, university_id=47)>]


In [21]:
@task("4")
def fourth():
    """
    Напишите запрос для получения списка студентов,
    проживающих в Воронеже и не получающих стипендию.
    """
    query: Query = db.session.query(
        Student
    ).filter(
        Student.city == 'Воронеж',
        or_(Student.stipend == 0, Student.stipend.is_(None))
    )
    pprint(query.all())
    
    
fourth()

4. Напишите запрос для получения списка студентов,
 проживающих в Воронеже и не получающих стипендию.

[<Student (first_name=Андрей, last_name=Павлов, stipend=0.0, course=3, city=Воронеж, birthdate=1979-11-05 00:00:00, university_id=10)>]


In [22]:
@task("5")
def fifth():
    """
    Напишите запрос для получения списка университетов,
    расположенных в Москве и имеющих рейтинг меньший, чем у НГУ.
    Значение рейтинга НГУ получите с помощью отдельного запроса или подзапроса.
    """
    ngu_rating = db.session.query(
        University.rating
    ).filter(
        University.name == 'НГУ'
    ).scalar()

    query: Query = db.session.query(
        University
    ).filter(
        University.city == 'Москва',
        University.rating < ngu_rating
    )

    pprint(query.all())
    
    
fifth()

5. Напишите запрос для получения списка университетов,
 расположенных в Москве и имеющих рейтинг меньший, чем у НГУ.
 Значение рейтинга НГУ получите с помощью отдельного запроса или подзапроса.

[<University (name=ФинУ, rating=330, city=Москва)>,
 <University (name=МТУСИ, rating=295, city=Москва)>]


In [23]:
@task("6")
def sixth():
    """
    Напишите запрос, выполняющий вывод находящихся в таблице
    EXAM_MARKS номеров предметов обучения, экзамены по
    которым сдавались между 1 и 21 марта 2020 г.
    """
    query: Query = db.session.query(
        ExamMark.subject_id, ExamMark.date
    ).filter(
        ExamMark.date.between(datetime(2020, 3, 1), datetime(2020, 3, 21))
    )
    pprint(query.all())
    
    
sixth()

6. Напишите запрос, выполняющий вывод находящихся в таблице
 EXAM_MARKS номеров предметов обучения, экзамены по
 которым сдавались между 1 и 21 марта 2020 г.

[(73, datetime.datetime(2020, 3, 20, 0, 0)),
 (12, datetime.datetime(2020, 3, 13, 0, 0)),
 (10, datetime.datetime(2020, 3, 21, 0, 0))]


In [24]:
@task("7")
def seventh():
    """
    Напишите запрос, который выполняет вывод названий
    предметов обучения, начинающихся на букву 'И'.
    """
    query: Query = db.session.query(
        Subject
    ).filter(
        Subject.name.startswith('И')
    )
    pprint(query.all())

    
seventh()

7. Напишите запрос, который выполняет вывод названий
 предметов обучения, начинающихся на букву 'И'.

[<Subject (name=Информатика, hour=56, semester=1)>,
 <Subject (name=История, hour=34, semester=4)>]


In [25]:
@task("8")
def eighth():
    """
    Напишите запрос, выбирающий сведения о студентах,
    у которых имена начинаются на букву 'И' или 'С'.
    """
    query: Query = db.session.query(
        Student
    ).filter(
        or_(Student.first_name.startswith('И'), Student.first_name.startswith('С'))
    )
    pprint(query.all())
    
    
eighth()

8. Напишите запрос, выбирающий сведения о студентах,
 у которых имена начинаются на букву 'И' или 'С'.

[<Student (first_name=Иван, last_name=Иванов, stipend=150.0, course=1, city=Орел, birthdate=1982-12-03 00:00:00, university_id=10)>,
 <Student (first_name=Софья, last_name=Дементьева, stipend=150.0, course=1, city=Нерчинск, birthdate=1996-09-25 00:00:00, university_id=15)>,
 <Student (first_name=Ия, last_name=Соколова, stipend=100.0, course=1, city=Колпашево, birthdate=1995-06-25 00:00:00, university_id=18)>,
 <Student (first_name=Ирина, last_name=Медведева, stipend=100.0, course=1, city=None, birthdate=2000-08-22 00:00:00, university_id=15)>,
 <Student (first_name=Синицин, last_name=Кондрат, stipend=200.0, course=4, city=Ишим, birthdate=1995-02-23 00:00:00, university_id=18)>,
 <Student (first_name=Светлана, last_name=Мишина, stipend=150.0, course=2, city=Рыльск, birthdate=1997-05-22 00:00:00, university_id=18)>,
 <Student (first_name=Ирина, last_name=Сорокина, stipend=200.0, course

In [26]:
@task("9")
def ninth():
    """
    Напишите запрос для получения списка предметов обучения,
    названия которых состоят из более одного слова.
    """
    query: Query = db.session.query(
        Subject
    ).filter(
        Subject.name.like('% %')
    )
    pprint(query.all())
    
    
ninth()

9. Напишите запрос для получения списка предметов обучения,
 названия которых состоят из более одного слова.

[<Subject (name=Анализ данных, hour=42, semester=1)>]


In [27]:
@task("10")
def tenth():
    """
    Напишите запрос для получения списка студентов,
    фамилии которых состоят из трех букв.
    """
    query: Query = db.session.query(
        Student
    ).filter(
        func.length(Student.last_name) == 3
    )
    pprint(query.all())
    
    
tenth()

10. Напишите запрос для получения списка студентов,
 фамилии которых состоят из трех букв.

[<Student (first_name=Бернар, last_name=Шоу, stipend=120.0, course=2, city=Ульяновск, birthdate=1979-08-05 00:00:00, university_id=44)>,
 <Student (first_name=Джон, last_name=Доу, stipend=120.0, course=2, city=Ульяновск, birthdate=1979-08-05 00:00:00, university_id=44)>]


In [28]:
@task("11")
def eleventh():
    """
    Составьте запрос для таблицы STUDENT таким образом,
    чтобы получить результат в следующем виде.
    Распечатайте первые 9 записей результата.

    И. Иванов   1982-12-03
    П. Петров   1980-12-01
    В. Сидоров  1979-06-07
    """
    query: Query = db.session.query(
        Student.first_name, Student.last_name, Student.birthdate
    ).limit(9)
    pprint([f"{i[0][0]}. {i[1]}   {datetime.strftime(i[2], '%Y-%m-%d')}" for i in query.all()])

    
eleventh()

11. Составьте запрос для таблицы STUDENT таким образом,
 чтобы получить результат в следующем виде.
 Распечатайте первые 9 записей результата.

 И. Иванов 1982-12-03
 П. Петров 1980-12-01
 В. Сидоров 1979-06-07

['И. Иванов   1982-12-03',
 'П. Петров   1980-12-01',
 'В. Сидоров   1979-06-07',
 'Б. Кузнецов   1981-12-08',
 'О. Зайцева   1981-05-01',
 'П. Котов   2021-02-28',
 'В. Белкин   1980-01-07',
 'Е. Сергеева   1997-07-04',
 'В. Кудряшова   2002-02-18']


In [29]:
@task("12")
def twelve():
    """
    Напишите запрос для получения списка студентов,
    фамилии которых начинаются на ‘Ков’ или на ‘Куз’.
    """
    query: Query = db.session.query(
        Student
    ).filter(
        or_(Student.last_name.startswith('Ков'), Student.last_name.startswith('Куз'))
    )
    pprint(query.all())
    
    
twelve()

12. Напишите запрос для получения списка студентов,
 фамилии которых начинаются на ‘Ков’ или на ‘Куз’.

[<Student (first_name=Борис, last_name=Кузнецов, stipend=0.0, course=2, city=Брянск, birthdate=1981-12-08 00:00:00, university_id=10)>,
 <Student (first_name=Ефим, last_name=Ковалев, stipend=200.0, course=2, city=Волгоград, birthdate=1995-11-28 00:00:00, university_id=47)>]


In [30]:
@task("13")
def thirteenth():
    """
    Напишите запрос для получения списка предметов,
    названия которых оканчиваются на ‘ия’.
    """
    query: Query = db.session.query(
        Subject
    ).filter(
        Subject.name.endswith('ия')
    )
    pprint(query.all())
    
    
thirteenth()

13. Напишите запрос для получения списка предметов,
 названия которых оканчиваются на ‘ия’.

[<Subject (name=История, hour=34, semester=4)>]


In [31]:
@task("14")
def fourteenth():
    """
    Напишите запрос для выбора из таблицы EXAM_MARKS записей,
    для которых отсутствуют значения оценок (поле MARK).
    """
    query: Query = db.session.query(
        ExamMark
    ).filter(
        ExamMark.mark.is_(None)
    )
    pprint(query.all())
    
    
fourteenth()

14. Напишите запрос для выбора из таблицы EXAM_MARKS записей,
 для которых отсутствуют значения оценок (поле MARK).

[<ExamMark (mark=None, date=1999-06-22 00:00:00, student_id=55, subject_id=10)>]


In [32]:
@task("15")
def fifteenth():
    """
    Составьте запрос, выводящий фамилии, имена студентов и величину получаемых ими стипендий,
    при этом значения стипендий должны быть увеличены в 100 раз.
    """
    query: Query = db.session.query(
        Student.last_name, Student.first_name, Student.stipend * 100
    )
    pprint(query.all())
    
    
fifteenth()

15. Составьте запрос, выводящий фамилии, имена студентов и величину получаемых ими стипендий,
 при этом значения стипендий должны быть увеличены в 100 раз.

[('Иванов', 'Иван', 15000.0),
 ('Петров', 'Петр', 20000.0),
 ('Сидоров', 'Вадим', 15000.0),
 ('Кузнецов', 'Борис', 0.0),
 ('Зайцева', 'Ольга', 25000.0),
 ('Котов', 'Павел', 15000.0),
 ('Белкин', 'Вадим', 25000.0),
 ('Сергеева', 'Елизавета', 15000.0),
 ('Кудряшова', 'Вера', 10000.0),
 ('Журавлева', 'Вера', 0.0),
 ('Дементьева', 'Софья', 15000.0),
 ('Рожкова', 'Пелагея', 0.0),
 ('Соколова', 'Ия', 10000.0),
 ('Семенова', 'Вероника', 0.0),
 ('Медведева', 'Ирина', 10000.0),
 ('Афанасьева', 'Оксана', 25000.0),
 ('Сергеева', 'Фёкла', 10000.0),
 ('Некрасова', 'Вероника', 25000.0),
 ('Игнатьева', 'Оксана', 20000.0),
 ('Казакова', 'Ольга', 25000.0),
 ('Шубина', 'Елена', 0.0),
 ('Миронова', 'Анна', 20000.0),
 ('Кондрат', 'Синицин', 20000.0),
 ('Пономарева', 'Алина', 25000.0),
 ('Одинцова', 'Лидия', 10000.0),
 ('Гуляева', 'Рукижат', 0.0),
 ('И

In [33]:
@task("16")
def sixteenth():
    """
    Составьте запрос для таблицы UNIVERSITY таким образом, чтобы выходная таблица
    содержала всего один столбец в следующем виде: Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.
    """
    query: Query = db.session.query(
        University.id, University.name, University.city, University.rating
    )
    pprint([f"Код-{i[0]}; {i[1]}-г.{i[2].upper()}; Рейтинг={i[3]}." for i in query.all()])
    
    
sixteenth()

16. Составьте запрос для таблицы UNIVERSITY таким образом, чтобы выходная таблица
 содержала всего один столбец в следующем виде: Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.

['Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.',
 'Код-11; НГУ-г.НОВОСИБИРСК; Рейтинг=345.',
 'Код-14; БГУ-г.БЕЛГОРОД; Рейтинг=326.',
 'Код-15; ТГУ-г.ТОМСК; Рейтинг=368.',
 'Код-18; ВГМА-г.ВОРОНЕЖ; Рейтинг=327.',
 'Код-22; МГУ-г.МОСКВА; Рейтинг=400.',
 'Код-32; РГУ-г.РОСТОВ; Рейтинг=416.',
 'Код-44; ФинУ-г.МОСКВА; Рейтинг=330.',
 'Код-45; МГТУ-г.МОСКВА; Рейтинг=372.',
 'Код-46; Политех-г.САНКТ-ПЕТЕРБУРГ; Рейтинг=300.',
 'Код-47; КФУ-г.КАЗАНЬ; Рейтинг=330.',
 'Код-48; УЛГУ-г.УЛЬЯНОВСК; Рейтинг=231.',
 'Код-49; МТУСИ-г.МОСКВА; Рейтинг=295.']


In [34]:
@task("17")
def seventeenth():
    """
    Напишите запрос для подсчета количества студентов,
    сдававших экзамен по предмету обучения с идентификатором 10.
    """
    query: Query = db.session.query(
        ExamMark
    ).filter(
        ExamMark.subject_id == 10
    )
    pprint(query.count())
    
    
seventeenth()

17. Напишите запрос для подсчета количества студентов,
 сдававших экзамен по предмету обучения с идентификатором 10.

10


In [35]:
@task("18")
def eighteenth():
    """
    Напишите запрос, который позволяет подсчитать в таблице
    EXAM_MARKS количество различных предметов обучения.
    """
    query: Query = db.session.query(
        ExamMark.subject_id
    ).distinct()
    pprint(query.count())
    
    
eighteenth()

18. Напишите запрос, который позволяет подсчитать в таблице
 EXAM_MARKS количество различных предметов обучения.

9


In [36]:
@task("19")
def nineteenth():
    """
    Напишите запрос, который для каждого студента выполняет
    выборку его идентификатора и минимальной из полученных им оценок.
    """
    query: Query = db.session.query(
        Student, func.min(ExamMark.mark)
    ).join(ExamMark).group_by(
        ExamMark.student_id
    )
    pprint(query.all())
    
    
nineteenth()

19. Напишите запрос, который для каждого студента выполняет
 выборку его идентификатора и минимальной из полученных им оценок.

[(<Student (first_name=Иван, last_name=Иванов, stipend=150.0, course=1, city=Орел, birthdate=1982-12-03 00:00:00, university_id=10)>,
  1),
 (<Student (first_name=Вадим, last_name=Сидоров, stipend=150.0, course=4, city=Москва, birthdate=1979-06-07 00:00:00, university_id=22)>,
  4),
 (<Student (first_name=Борис, last_name=Кузнецов, stipend=0.0, course=2, city=Брянск, birthdate=1981-12-08 00:00:00, university_id=10)>,
  5),
 (<Student (first_name=Ольга, last_name=Зайцева, stipend=250.0, course=2, city=Липецк, birthdate=1981-05-01 00:00:00, university_id=10)>,
  5),
 (<Student (first_name=Павел, last_name=Котов, stipend=150.0, course=5, city=Белгород, birthdate=2021-02-28 00:00:00, university_id=14)>,
  4),
 (<Student (first_name=Вадим, last_name=Белкин, stipend=250.0, course=5, city=Воронеж, birthdate=1980-01-07 00:00:00, university_id=10)>,
  5),
 (<Student (f

In [37]:
@task("20")
def twentieth():
    """
    Напишите запрос, который для каждого предмета обучения выводит
    наименование предмета и максимальное значение номера семестра,
    в котором этот предмет преподается.
    """
    query: Query = db.session.query(
        Subject.name, func.max(Subject.semester)
    ).group_by(
        Subject.name
    )
    pprint(query.all())
    
    
twentieth()

20. Напишите запрос, который для каждого предмета обучения выводит
 наименование предмета и максимальное значение номера семестра,
 в котором этот предмет преподается.

[('Анализ данных', 1),
 ('Английский', 3),
 ('Информатика', 1),
 ('История', 4),
 ('Математика', 2),
 ('ОБЖ', 2),
 ('Программирование', 2),
 ('Физика', 1),
 ('Физкультура', 5)]


In [39]:
@task("21")
def twenty_first():
    """
    Напишите запрос, который для каждого конкретного дня сдачи экзамена
    выводит данные о количестве студентов, сдававших экзамен в этот день.
    """
    query: Query = db.session.query(
        ExamMark.date, func.count(ExamMark.student_id)
    ).group_by(
        ExamMark.date
    )
    pprint(query.all())
    
    
twenty_first()

21. Напишите запрос, который для каждого конкретного дня сдачи экзамена
 выводит данные о количестве студентов, сдававших экзамен в этот день.

[(datetime.datetime(1999, 6, 17, 0, 0), 1),
 (datetime.datetime(1999, 6, 22, 0, 0), 1),
 (datetime.datetime(2000, 1, 5, 0, 0), 1),
 (datetime.datetime(2000, 1, 18, 0, 0), 1),
 (datetime.datetime(2000, 1, 23, 0, 0), 1),
 (datetime.datetime(2006, 1, 12, 0, 0), 1),
 (datetime.datetime(2019, 5, 11, 0, 0), 1),
 (datetime.datetime(2019, 6, 8, 0, 0), 1),
 (datetime.datetime(2019, 7, 4, 0, 0), 1),
 (datetime.datetime(2019, 7, 6, 0, 0), 1),
 (datetime.datetime(2019, 7, 24, 0, 0), 1),
 (datetime.datetime(2019, 8, 14, 0, 0), 1),
 (datetime.datetime(2019, 8, 30, 0, 0), 1),
 (datetime.datetime(2019, 9, 2, 0, 0), 1),
 (datetime.datetime(2019, 10, 28, 0, 0), 1),
 (datetime.datetime(2019, 12, 17, 0, 0), 1),
 (datetime.datetime(2019, 12, 25, 0, 0), 1),
 (datetime.datetime(2019, 12, 31, 0, 0), 1),
 (datetime.datetime(2020, 2, 10, 0, 0), 1),
 (datetime.datetime(2

In [40]:
@task("22")
def twenty_second():
    """
    Напишите запрос, выдающий средний балл для каждого студента.
    """
    query: Query = db.session.query(
        Student, func.avg(ExamMark.mark)
    ).join(ExamMark).group_by(
        ExamMark.student_id
    )
    pprint(query.all())
    
    
twenty_second()

22. Напишите запрос, выдающий средний балл для каждого студента.

[(<Student (first_name=Иван, last_name=Иванов, stipend=150.0, course=1, city=Орел, birthdate=1982-12-03 00:00:00, university_id=10)>,
  1.0),
 (<Student (first_name=Вадим, last_name=Сидоров, stipend=150.0, course=4, city=Москва, birthdate=1979-06-07 00:00:00, university_id=22)>,
  4.0),
 (<Student (first_name=Борис, last_name=Кузнецов, stipend=0.0, course=2, city=Брянск, birthdate=1981-12-08 00:00:00, university_id=10)>,
  5.0),
 (<Student (first_name=Ольга, last_name=Зайцева, stipend=250.0, course=2, city=Липецк, birthdate=1981-05-01 00:00:00, university_id=10)>,
  5.0),
 (<Student (first_name=Павел, last_name=Котов, stipend=150.0, course=5, city=Белгород, birthdate=2021-02-28 00:00:00, university_id=14)>,
  4.0),
 (<Student (first_name=Вадим, last_name=Белкин, stipend=250.0, course=5, city=Воронеж, birthdate=1980-01-07 00:00:00, university_id=10)>,
  5.0),
 (<Student (first_name=Вероника, last_name=Семенова, stipend=0.

In [42]:
@task("23")
def twenty_third():
    """
    Напишите запрос, выдающий средний балл для каждого экзамена.
    """
    query: Query = db.session.query(ExamMark)
    pprint(query.all())
    
    
twenty_third()

23. Напишите запрос, выдающий средний балл для каждого экзамена.

[<ExamMark (mark=1, date=2020-07-26 00:00:00, student_id=97, subject_id=10)>,
 <ExamMark (mark=4, date=2020-12-23 00:00:00, student_id=85, subject_id=73)>,
 <ExamMark (mark=4, date=2020-03-20 00:00:00, student_id=64, subject_id=73)>,
 <ExamMark (mark=1, date=2020-08-10 00:00:00, student_id=126, subject_id=56)>,
 <ExamMark (mark=3, date=2020-05-10 00:00:00, student_id=71, subject_id=10)>,
 <ExamMark (mark=4, date=2020-04-06 00:00:00, student_id=117, subject_id=56)>,
 <ExamMark (mark=1, date=2021-02-10 00:00:00, student_id=79, subject_id=12)>,
 <ExamMark (mark=1, date=2020-03-13 00:00:00, student_id=116, subject_id=12)>,
 <ExamMark (mark=5, date=2020-02-22 00:00:00, student_id=91, subject_id=22)>,
 <ExamMark (mark=1, date=2019-12-25 00:00:00, student_id=110, subject_id=22)>,
 <ExamMark (mark=2, date=2020-12-04 00:00:00, student_id=76, subject_id=56)>,
 <ExamMark (mark=5, date=2019-09-02 00:00:00, student_id=88, subject_id=